# Conversational Interface - Chatbot with Claude LLM

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

In this notebook, we will build a chatbot using the Foundation Models (FMs) in Amazon Bedrock. For our use-case we use Claude as our FM for building the chatbot.

## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

%pip install --no-build-isolation --force-reinstall \
    "boto3>1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

In this notebook, we'll also need some extra dependencies:

- [FAISS](https://github.com/facebookresearch/faiss), to store vector embeddings
- [IPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), for interactive UI widgets in the notebook
- [PyPDF](https://pypi.org/project/pypdf/), for handling PDF files

%pip install --quiet "faiss-cpu>=1.7,<2" langchain==0.0.304 "pypdf>=3.8,<4"
%pip install --upgrade sqlalchemy

In [2]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::195364414018:role/Crossaccountbedrock"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-west-2
  Using role: arn:aws:iam::195364414018:role/Crossaccountbedrock ... successful!
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [3]:
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)
cl_llm = Bedrock(model_id="anthropic.claude-v2",client=boto3_bedrock)

#### FAISS as VectorStore

In order to be able to use embeddings for search, we need a store that can efficiently perform vector similarity searches. In this notebook we use FAISS, which is an in memory store. For permanently store vectors, one can use pgVector, Pinecone or Chroma.

The langchain VectorStore API's are available [here](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html)

To know more about the FAISS vector store please refer to this [document](https://arxiv.org/pdf/1702.08734.pdf).

In [4]:
from langchain.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

loader = PyPDFLoader("policy_certifcate_multiple_vehicle.pdf")
pages = loader.load()

chunk_size = 1000
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=100,
    length_function = len,
)

docs, metadata = [], []

for i in range(len(pages)):
    print(f"Spliting the content with length", len(pages[i].page_content))
    splits = text_splitter.split_text(pages[i].page_content)
    docs.extend(splits)
    metadata.extend([{"source": pages[i].metadata["source"]}] * len(splits))

vectorstore_faiss_aws = FAISS.from_texts(
    docs,
    br_embeddings,
    metadatas=metadata,
)

print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}")
# with open("vectorstore_faiss_aws.pkl", "wb") as f:
#     pickle.dump(vectorstore_faiss_aws, f)
    


Spliting the content with length 2579
Spliting the content with length 85
Spliting the content with length 1269
Spliting the content with length 3463
Spliting the content with length 85
Spliting the content with length 1295
Spliting the content with length 85
Spliting the content with length 2567
Spliting the content with length 708
Spliting the content with length 2789
Spliting the content with length 2121
Spliting the content with length 819
Spliting the content with length 85
Spliting the content with length 2850
Spliting the content with length 2124
Spliting the content with length 2851
Spliting the content with length 2124
vectorstore_faiss_aws: number of elements in the index=41


#### Semantic search

We can use a Wrapper class provided by LangChain to query the vector data base store and return to us the relevant documents. Behind the scenes this is only going to run a RetrievalQA chain.

In [5]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print_ww(wrapper_store_faiss.query("Account name of the policy?", llm=cl_llm))

 Based on the policy details provided, the name of the account holder/policyholder is Mr. Test
Tester. The policy schedule refers to "Your policy schedule" and "Vehicle policyholder: Mr. Test
Tester" for the two vehicles listed. So the account name appears to be Mr. Test Tester.


Let's see how the semantic search works:
1. First we calculate the embeddings vector for the query, and
2. then we use this vector to do a similarity search on the store

In [6]:
v = br_embeddings.embed_query("Account name of the policy?")
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=4)
for r in results:
    print_ww(r.page_content)
    print('----')

[-0.7109375, -0.5859375, 0.30859375, -0.33984375, 0.62109375, 7.104874e-05, 0.08203125, -0.00032424927, 0.13867188, 0.43164062]
1598Petrol
Automatic3DoorHatchback
Vehicle
policyholder:Mr.TestTester
Drivingoption:Vehiclepolicyholderonly
-Thisvehiclehascomprehensivecoverbasedon£450excessandupto8000milesayearperyearofSocial,Domestic,Plea
sureand
Commutingandrestrictedbusinessuse(refertothevehiclecertificate).
-Thevehiclesareorwillbeownedandregisteredbyeitheryou,yourspouse/civil/domesticpartner,acloserelative
residingatthesame
address,yourcompanyoristhesubjectofaprivateorpersonalleasingcontract.
-Vehiclemodifications:nomodifications
-Therearenoadditionaldrivers.
-Coverforthisvehicleisbasedon0yearsnoclaimsdiscount(NCD)whichisunprotected.FutureNCDwillbeownedbyMr.
Test
Tester.-*- Demonstration Powered by OpenText Exstream 09/27/2023, Version 16.6.32 32-bit -*-
----
AvivaInsuranceLimited.RegisteredinScotland,No.2116.RegisteredOffice:Pitheavlis,PerthPH20NH.Authorise
dbythePrudentialRegulationAu

#### Memory
In any chatbot we will need a QA Chain with various options which are customized by the use case. But in a chatbot we will always need to keep the history of the conversation so the model can take it into consideration to provide the answer. In this example we use the [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_vector_db) from LangChain, together with a ConversationBufferMemory to keep the history of the conversation.

Source: https://python.langchain.com/docs/modules/chains/popular/chat_vector_db

Set `verbose` to `True` to see all the what is going on behind the scenes.

In [7]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

print_ww(CONDENSE_QUESTION_PROMPT.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a
standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


#### Parameters used for ConversationRetrievalChain
* **retriever**: We used `VectorStoreRetriever`, which is backed by a `VectorStore`. To retrieve text, there are two search types you can choose: `"similarity"` or `"mmr"`. `search_type="similarity"` uses similarity search in the retriever object where it selects text chunk vectors that are most similar to the question vector.

* **memory**: Memory Chain to store the history 

* **condense_question_prompt**: Given a question from the user, we use the previous conversation and that question to make up a standalone question

* **chain_type**: If the chat history is long and doesn't fit the context you use this parameter and the options are `stuff`, `refine`, `map_reduce`, `map-rerank`

If the question asked is outside the scope of context, then the model will reply it doesn't know the answer

**Note**: if you are curious how the chain works, uncomment the `verbose=True` line.

In [8]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.chat_memory.add_user_message("You will be acting as a Insurance Assistant for Aviva an insurance company. Your goal is to provide correct and valid answer for user queries.")
memory.chat_memory.add_ai_message("Ok, I am a insurance assistant for aviva and give proper answer for customer queries.")


qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    memory=memory,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    verbose=True, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

In [13]:
result = qa({"question": "How much is the third party damage limit?"})
result



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: You will be acting as a Insurance Assistant for Aviva an insurance company. Your goal is to provide correct and valid answer for user queries.
Assistant: Ok, I am a insurance assistant for aviva and give proper answer for customer queries.
Human: Account name of the policy?
Assistant:  Based on the policy schedule provided, the account name for this policy is Mr. Test Tester. The schedule mentions "Your policy schedule-continued" and "policyholder:Mr.TestTester" for the vehicles insured under this policy.
Human: What is my policy number?
Assistant:  Based on the policy documents provided, Mr. Test Tester's Aviva auto insurance policy number is MMV070055371.
Follow Up Input: How much is the third party damage limit?
Standalone question:

> Finished chain.


> En

{'question': 'How much is the third party damage limit?',
 'chat_history': [HumanMessage(content='You will be acting as a Insurance Assistant for Aviva an insurance company. Your goal is to provide correct and valid answer for user queries.', additional_kwargs={}, example=False),
  AIMessage(content='Ok, I am a insurance assistant for aviva and give proper answer for customer queries.', additional_kwargs={}, example=False),
  HumanMessage(content='Account name of the policy?', additional_kwargs={}, example=False),
  AIMessage(content=' Based on the policy schedule provided, the account name for this policy is Mr. Test Tester. The schedule mentions "Your policy schedule-continued" and "policyholder:Mr.TestTester" for the vehicles insured under this policy.', additional_kwargs={}, example=False),
  HumanMessage(content='What is my policy number?', additional_kwargs={}, example=False),
  AIMessage(content=" Based on the policy documents provided, Mr. Test Tester's Aviva auto insurance pol

In [10]:
result["answer"]

' Based on the policy schedule provided, the account name for this policy is Mr. Test Tester. The schedule mentions "Your policy schedule-continued" and "policyholder:Mr.TestTester" for the vehicles insured under this policy.'

In [11]:
result = qa({"question": "What is my policy number?"})
result



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: You will be acting as a Insurance Assistant for Aviva an insurance company. Your goal is to provide correct and valid answer for user queries.
Assistant: Ok, I am a insurance assistant for aviva and give proper answer for customer queries.
Human: Account name of the policy?
Assistant:  Based on the policy schedule provided, the account name for this policy is Mr. Test Tester. The schedule mentions "Your policy schedule-continued" and "policyholder:Mr.TestTester" for the vehicles insured under this policy.
Follow Up Input: What is my policy number?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. 

{'question': 'What is my policy number?',
 'chat_history': [HumanMessage(content='You will be acting as a Insurance Assistant for Aviva an insurance company. Your goal is to provide correct and valid answer for user queries.', additional_kwargs={}, example=False),
  AIMessage(content='Ok, I am a insurance assistant for aviva and give proper answer for customer queries.', additional_kwargs={}, example=False),
  HumanMessage(content='Account name of the policy?', additional_kwargs={}, example=False),
  AIMessage(content=' Based on the policy schedule provided, the account name for this policy is Mr. Test Tester. The schedule mentions "Your policy schedule-continued" and "policyholder:Mr.TestTester" for the vehicles insured under this policy.', additional_kwargs={}, example=False),
  HumanMessage(content='What is my policy number?', additional_kwargs={}, example=False),
  AIMessage(content=" Based on the policy documents provided, Mr. Test Tester's Aviva auto insurance policy number is MM

In [12]:
result["answer"]

" Based on the policy documents provided, Mr. Test Tester's Aviva auto insurance policy number is MMV070055371."